## Project sentiment stock

### Objective:
- In this project, we will perform a sentiment analysis of data published on X (Twitter), in real time, relating to stock market quotes. This analysis can be passed on to our investors, through sentiment percentages, whether the messages about the quotes present a positive or negative trend for the subject in question.

- Nesse projeto vamos fazer uma análise de sentimento de dados publicados no X (twitter), em tempo real, relativo às cotações da bolsa de valores. Essa análise vai poder passar para nossos investidores, através dos percentuais de sentimento, se as postagens sobre as cotações, apresentam uma tendência positiva ou negativa para o assunto tratado.

### Data Origin:
- Dataset: financial_tweets.csv

- This file was acquired by capturing tweets from users who made publications within the subject discussed here, expressing their feelings about stock market quotes in real time. The data is intercepted through an API provided by X (twitter) and saved in a .csv file.

- Esse arquivo foi adquirido através da captura de tweets dos usuários que fizeram publicações dentro do assunto aqui tratado, expressando seus sentimentos sobre cotações da bolsa em tempo real. Os dados são interceptados através de API fornecida pelo X (twitter) e gravados em arquivo .csv.

Aqui o que nos interessa é somente o campo:
- Description: descrição do sentimento publicado pelo usuário.

- ## Análise exploratória dos dados
- ## Preparação dos dados
- ## Armazenamento dos dados tratados
- ## Geração de sentimento
    - textblob
    - vader
    - bert

In [2]:
# maximiza nro de linhas e colunas para exibição
# inibe mensagens de warning
import pandas as pd
pd.set_option('display.max_rows', None) # permite a máxima visualização das linhas em um display
pd.set_option('display.max_columns', None) # permite a máxima visualização das colunas em um display
import warnings
warnings.simplefilter('ignore') # inibe a exibição de avisos de warning

In [3]:
# bibliotecas
import tweepy as tw
import numpy as np # computação de matrizes

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from textblob import TextBlob

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

import tensorflow as tf

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\KR\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
consumer_key = 'bokA1eIxmcK5y396klmyThyxE'
consumer_secret = 'kpI1wT4So74kl6OVbv10rMUk5xkPNJZhkCCPScf5cyzHjLCJ62'
access_token = '2689849668-kaDvfg15xpeF4a1D6pL9ck1EnYTQhBjGklcS77S'
access_token_secret = 'zPdg70A5cnkmmXmuvM62TlkxoHl4IHJGpGfD3c5stfday'

In [6]:
autorizacao = tw.OAuthHandler(consumer_key, consumer_secret)

In [7]:
autorizacao.set_access_token(access_token, access_token_secret)

In [8]:
api = tw.API(autorizacao)

In [ ]:
meus_tweets = api.home_timeline() # retorna uma coleção dos Tweets e Retweets mais recentes postados pelo usuário autenticado e pelos usuários que ele segue.

In [ ]:
for twitters in meus_tweets:
    print(twitters.created_at)
    print(twitters.text,  '\n')

In [ ]:
query = "bitcoin AND price"
pesquisa = api.search(q=query,lang='en',tweet_mode="extended",count=50)

In [ ]:
resultado = []

for twitters in pesquisa:
  resultado.append(twitters.created_at)
  resultado.append(twitters.full_text)

In [ ]:
matriz_np = np.array(resultado)
matriz_ajustada = np.reshape(matriz_np, (50,2))

In [ ]:
df = pd.DataFrame()

colunas = [
    'Data_publicacao',
    'tweet'
]

df = pd.DataFrame(matriz_ajustada,columns=colunas)

In [ ]:
arquivo_tweets = 'dataset/financial_tweets.csv'
df.to_csv(arquivo_tweets, encoding='utf-8', index=False,quotechar='|')

In [4]:
# carregando arquivo tratado em dataframe
df_sentiment = pd.read_csv('dataset/financial_tweets.csv', sep=',')

In [5]:
df = df_sentiment

In [6]:
df = df[:2000]

In [7]:
df.shape

(2000, 13)

In [8]:
df = df['description']

In [9]:
df.head(5)

0    Crazy that $PRIME is going to >$1B market cap ...
1    Crazy part is, a lot of what I have been tradi...
2                       $AVAX\n\nStacking on support..
3                                        $SOL\n\nLFG!!
4    $INJ - Twitter been super bullish on this one ...
Name: description, dtype: object

In [10]:
english_stopwords = stopwords.words("english")
punctuation_words = string.punctuation

In [11]:
tweet_list = df.tolist()

In [12]:
clean_tweets = []

for tweet in tweet_list:
    
    tweet = str(tweet) 

    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'$', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'https?://[A-Za-z0-9./]+','',tweet)
    tweet = re.sub(r'https//[A-Za-z0-9./]+','',tweet)
    tweet = tweet.replace("\n","")
    tweet = re.sub('@[\w]+', '', tweet)

    tweet = re.sub(r"n't", " not ", tweet)
    tweet = re.sub(r"\'s", " ", tweet)
    tweet = re.sub(r"\'ve", " have ", tweet)
    tweet = re.sub(r"\'re", " are ", tweet)
    tweet = re.sub(r"\'d", " would ", tweet)
    tweet = re.sub(r"\'ll", " will ", tweet) 

    tweet = tweet.lower()
    tweet = tweet.strip()

    clean_tweets.append(tweet)

In [13]:
clean_tweets[:4]

['crazy that  is going to > market cap at the next cycle top as the “  of this cycle” and you still don’t own any anon. -100 exit or bust bitch.',
 'crazy part is, a lot of what i have been trading is still up..',
 'stacking on support..',
 'lfg!!']

In [14]:
df = pd.DataFrame(clean_tweets)

In [15]:
df.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [18]:
df_tblob = df.copy()

In [19]:
df_vader = df.copy()

In [20]:
df_bert = df.copy()

In [21]:
df_tblob.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [22]:
for row in df_tblob.itertuples():
    tweet = df_tblob.at[row[0],0]
    analysis = TextBlob(tweet)
    df_tblob.at[row[0], 'polarity'] = analysis.sentiment[0]
    df_tblob.at[row[0], 'subjectivity'] = analysis.sentiment[1]
    if analysis.sentiment[0]>0:
        df_tblob.at[row[0], 'Sentiment'] = "Positivo"
    elif analysis.sentiment[0]<0:
        df_tblob.at[row[0], 'Sentiment'] = "Negativo"
    else:
        df_tblob.at[row[0], 'Sentiment'] = "Neutro"

In [23]:
df_tblob.head(5)

,0,polarity,subjectivity,Sentiment
0,crazy that is going to > market cap at the ne...,0.125000,0.600,Positivo
1,"crazy part is, a lot of what i have been tradi...",-0.600000,0.900,Negativo
2,stacking on support..,0.000000,0.000,Neutro
3,lfg!!,0.000000,0.000,Neutro
4,- twitter been super bullish on this one and t...,0.195833,0.325,Positivo


In [24]:
for index, row in df_vader[0].items():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    if score['neg'] > score['pos']:
        df_vader.loc[index, "Sentiment"] = "Negativo"
    elif score['pos'] > score['neg']:
        df_vader.loc[index, "Sentiment"] = "Positivo"
    else:
        df_vader.loc[index, "Sentiment"] = "Neutro"
        
    df_vader.loc[index, 'neg'] = score['neg']
    df_vader.loc[index, 'neu'] = score['neu']
    df_vader.loc[index, 'pos'] = score['pos']
    df_vader.loc[index, 'compound'] = score['compound']

In [25]:
df_vader.head(5)

,0,Sentiment,neg,neu,pos,compound
0,crazy that is going to > market cap at the ne...,Negativo,0.182,0.765,0.053,-0.6597
1,"crazy part is, a lot of what i have been tradi...",Negativo,0.179,0.821,0.000,-0.3400
2,stacking on support..,Neutro,0.000,1.000,0.000,0.0000
3,lfg!!,Neutro,0.000,1.000,0.000,0.0000
4,- twitter been super bullish on this one and t...,Positivo,0.000,0.809,0.191,0.7650


In [26]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
df_bert.shape

(2000, 1)

In [29]:
df_bert.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [30]:
df_bert[0] = df_bert[0].astype("str")

In [31]:
for index, row  in df_bert[0].items():
    score = sentiment_score(row)
    if score > 3:
        df_bert.loc[index, "Sentiment"] = "Positivo"
    elif score < 3:
        df_bert.loc[index, "Sentiment"] = "Negativo"
    else:
        df_bert.loc[index, "Sentiment"] = "Neutro"

In [32]:
df_bert.head(5)

,0,Sentiment
0,crazy that is going to > market cap at the ne...,Negativo
1,"crazy part is, a lot of what i have been tradi...",Neutro
2,stacking on support..,Neutro
3,lfg!!,Negativo
4,- twitter been super bullish on this one and t...,Positivo


In [33]:
# salva os dados tratados em arquivo csv organizado pelo índice
df_tblob.sort_index().to_csv('treated/df_tblob.csv', index=False)
df_vader.sort_index().to_csv('treated/df_vader.csv', index=False)
df_bert.sort_index().to_csv('treated/df_bert.csv', index=False)